### Venmo Python API

This notebook goes through some of the basic usage of the venmo_api with python for usage with networking. More documentation available [here]('https://venmo.readthedocs.io/en/latest/').

In [1]:
from venmo_api import Client 
import config #local file listing users venmo credentials

In [2]:
#The app must be logged in with a unique access_token. If you know your device_id you can skip two factor authentication
#The access_token does not expire! It can be used indefinitely but unless is saved locally it is better to logout. 
#again but is best practice to logout
access_token = Client.get_access_token(username = config.username,
                                        password = config.password, device_id = config.device_id)



IMPORTANT: Take a note of your device id to avoid 2-Factor-Authentication for your next login.
device-id: 86354220-15D2-2O45-00U6-3WJ02V247RQ9
IMPORTANT: Your Access Token will never expire, unless you logout using it. Take a note of it for your future use or even for logging out, you will need it.

Successfully logged in.
access_token: 2011160477e8f76900831a182817c3e57588c585a1005ca929b9c38880dd4653


In [3]:
#This will login to the venmo client using the access_token 
venmo = Client(access_token = access_token)

In [8]:
#This will logout of the client
venmo.log_out(f"Bearer {access_token}")

Successfully logged out.


True

In [4]:
#Gets your user object
user = venmo.user.get_my_profile()

In [7]:
#Example attributes
user.id, user.username

('1350836974256128854', 'Mike-Sender')

In [9]:
#This function returns the 50 last transactions
transactions = venmo.user.get_user_transactions(user_id = user.id)
#Prints the usernames of each transaction
[(transaction.actor.username, transaction.target.username) for transaction in transactions]

[('Wes-Wong', 'Mike-Sender'),
 ('Mike-Sender', 'Rachel-Payne-2'),
 ('Rachel-Sender', 'Mike-Sender'),
 ('JoshSender', 'Mike-Sender'),
 ('Mike-Sender', 'Ben-Sender'),
 ('Mike-Sender', 'Ilya-Yudkovich'),
 ('Anna-Membrino', 'Mike-Sender'),
 ('Mike-Sender', 'david_wang'),
 ('Wes-Wong', 'Mike-Sender'),
 ('Anna-Membrino', 'Mike-Sender'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Cassandra-Wong-4', 'Mike-Sender'),
 ('Anna-Membrino', 'Mike-Sender'),
 ('Anna-Membrino', 'Mike-Sender'),
 ('Anna-Membrino', 'Mike-Sender'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'kerrychang'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'Katie-Talay'),
 ('Brittany-Kaplan-4', 'Mike-Sender'),
 ('Rachel-Payne-2', 'Mike-Sender'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'Anna-Membrino'),
 ('Mike-Sender', 'Alex-Dagley'),
 ('JoshSender', 'Mike-Sender'),
 ('Rachel-Sender', 'Mike-Sender'),
 ('Ben-Sender', 'Mike-Sender'),
 ('Anna

In [101]:
import networkx as nx #network analysis package

G3 = nx.MultiGraph()




In [61]:
def neighbors_graph(user_id, G, levels):
    '''Create a graph of venmo transactions from a given user_id
    
    user_id = venmo.user.id
    G = nx.MultiGraph
    levels = distance from the initial user_id to generate
    
    '''
       
    user_levels = {user_id}
    nodes = {user_id}
    
    for level in range(levels):
        
                   
        for x in user_levels:
            print(x)
            try:
                transactions = venmo.user.get_user_transactions(user_id = x)
                
                G.add_edges_from(((transaction.actor.id, transaction.target.id) for transaction in transactions))
            
                nodes = nodes.union({transaction.target.id for transaction in transactions}.union(
                                {transaction.actor.id for transaction in transactions}))
               
                G.add_nodes_from(nodes)
            except:
                
                pass
            
        user_levels = nodes.difference(user_levels)

    


In [102]:
neighbors_graph(user.id, G3, 3)

In [73]:
personalization ={user.id:1}

In [142]:
ranked2 = nx.pagerank_numpy(G2, personalization = personalization)

In [143]:
ranked2 = {venmo.user.get_user(k).username : v for k, v in sorted(ranked2.items(), key=lambda item: item[1], reverse = True)}

In [136]:
ranked2['1608773021990912422']

0.00217765346818369

In [14]:
#The set of all users in the above transactions
level_1_users = {transaction.target.id for transaction in transactions if transaction.target.id != user.id}.union(
                {transaction.actor.id for transaction in transactions if transaction.actor.id != user.id})
                                    

In [17]:
#Functions which automates above cells for 
def neighbors(user_id):
    #Get through error if get_user_transactions runs into an error        
    try:
        transactions = venmo.user.get_user_transactions(user_id = user_id)
        new_users = {transaction.target.id for transaction in transactions if transaction.target.id != user_id}.union(\
                {transaction.actor.id for transaction in transactions if transaction.actor.id != user_id})
    except:
        #show the user_id which raised an error
        print(user_id)
        new_users = []
    return new_users
        
        

In [12]:
#Find all the users within 2 transactions from the original user... (me)
nodes = level_1_users
for user_id in level_1_users:
    if user_id != None:
        nodes = nodes.union(neighbors(user_id))
    

        

1539091850067968476


In [26]:
def callback(transactions_list):
    for transaction in transactions_list:
        try:
            print(transaction)
        except:
            pass
            

In [29]:
#error raising id for debugging: 1539091850067968476
transactions = venmo.user.get_user_transactions(user_id = '1539091850067968476',callback = callback)

Exception in thread Thread-7:
ValueError: 'disbursement' is not a valid TransactionType

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\msend\appdata\local\programs\python\python38-32\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\msend\appdata\local\programs\python\python38-32\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\msend\appdata\local\programs\python\python38-32\lib\site-packages\venmo_api\utils\api_client.py", line 110, in __call_api
    callback(processed_response)
  File "c:\users\msend\appdata\local\programs\python\python38-32\lib\site-packages\venmo_api\utils\api_util.py", line 63, in wrapper
    deserialized_data = deserialize(response=response, data_type=data_type, nested_response=nested_response)
  File "c:\users\msend\appdata\local\programs\python\python38-32\lib\site-packages\venmo_api\utils\api_util.py", line

In [30]:
transactions

In [18]:
#3 transactions from me
nodes2 = nodes.copy()
for user_id in nodes2:
    if user_id != None:
        nodes = nodes.union(neighbors(user_id))
    

1481509928173568655
913641482223616436
1504694404907008443
1536217610977280482
1410483659735040880
1485696070057984261
1172857195331584019
442240501547008808
1518391609262080730
910789783322624812
1476088664424449002
2211931268055040282
2577995902484480887
1416214219849728255
1058956877234176589
1376437529477120827
1539091850067968476


In [19]:
len(nodes)

9766